In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import pprint as pprint
import os

from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder

from sqlalchemy import create_engine

postdb = 'postgresql://postgres:' + os.getenv('PGPWD') + '@localhost:5432/nba'
engine = create_engine(postdb)

#api_key = "d1b1f11391mshdfeb577395264e5p18b5d8jsn5ad94c991841"
#url = "https://api-nba-v1.p.rapidapi.com/games/seasonYear/2017"

con = engine.connect()

In [2]:
# Output File (CSV)
nba_file = pd.read_csv('2017_nba_season_stats.csv')
nba_file.rename(columns={'Year':'season', 'Player': 'player', 'Pos':'position','G':'games','Age':'age','PTS':'points','Minutes Played':'minutes_played','FT%':'free_throw_pc','TRB':'rebounds','AST':'assists','STL':'steals','BLK':'blocks','TOV':'turnovers'}, inplace=True)

nba_file.head()


,season,player,position,age,Tm,games,GS,points,minutes_played,PER,...,free_throw_pc,ORB,DRB,rebounds,assists,steals,blocks,turnovers,PF,PTS.1
0,2017,Alex Abrines,SG,23,OKC,68,6,406,1055,10.1,...,0.898,18,68,86,40,37,8,33,114,406
1,2017,Quincy Acy,PF,26,TOT,38,1,222,558,11.8,...,0.750,20,95,115,18,14,15,21,67,222
2,2017,Quincy Acy,PF,26,DAL,6,0,13,48,-1.4,...,0.667,2,6,8,0,0,0,2,9,13
3,2017,Quincy Acy,PF,26,BRK,32,1,209,510,13.1,...,0.754,18,89,107,18,14,15,19,58,209
4,2017,Steven Adams,C,23,OKC,80,80,905,2389,16.5,...,0.611,282,333,615,86,88,78,146,195,905


In [3]:
#derive players

nba_notot = nba_file.loc[nba_file['Tm'] != 'TOT', ['player', 'position']]

nba_players=nba_notot.drop_duplicates(subset=['player','position'])
nba_players.reset_index(drop=True, inplace=True)


nba_players.head()


,player,position
0,Alex Abrines,SG
1,Quincy Acy,PF
2,Steven Adams,C
3,Arron Afflalo,SG
4,Alexis Ajinca,C


In [4]:
con.execute("Truncate players CASCADE")

nba_players.to_sql('players', con, if_exists='append', index_label='player_id')

In [5]:
nba_teams = pd.DataFrame(teams.get_teams())

nba_teams.rename(columns={'abbreviation': 'team_abbr', 'city':'team_city','full_name':'team_name','id':'team_id'}, inplace=True)

nba_teams.head()


,team_abbr,team_city,team_name,team_id,nickname,state,year_founded
0,ATL,Atlanta,Atlanta Hawks,1610612737,Hawks,Atlanta,1949
1,BOS,Boston,Boston Celtics,1610612738,Celtics,Massachusetts,1946
2,CLE,Cleveland,Cleveland Cavaliers,1610612739,Cavaliers,Ohio,1970
3,NOP,New Orleans,New Orleans Pelicans,1610612740,Pelicans,Louisiana,2002
4,CHI,Chicago,Chicago Bulls,1610612741,Bulls,Illinois,1966


In [6]:
con.execute("Truncate teams CASCADE")

nba_teams.to_sql('teams', con, if_exists='append', index=False)

In [7]:
pg_nba_players = pd.read_sql_table('players', con)

pg_nba_players.head()


,player_id,player,position
0,0,Alex Abrines,SG
1,1,Quincy Acy,PF
2,2,Steven Adams,C
3,3,Arron Afflalo,SG
4,4,Alexis Ajinca,C


In [8]:
pg_nba_teams = pd.read_sql_table('teams', con)

pg_nba_teams.head()

,team_abbr,team_city,team_name,team_id,nickname,state,year_founded
0,ATL,Atlanta,Atlanta Hawks,1610612737,Hawks,Atlanta,1949
1,BOS,Boston,Boston Celtics,1610612738,Celtics,Massachusetts,1946
2,CLE,Cleveland,Cleveland Cavaliers,1610612739,Cavaliers,Ohio,1970
3,NOP,New Orleans,New Orleans Pelicans,1610612740,Pelicans,Louisiana,2002
4,CHI,Chicago,Chicago Bulls,1610612741,Bulls,Illinois,1966


In [9]:
#  Retrieve Teams

nba_player_stats_player = pd.merge(nba_file, pg_nba_players, on='player')

nba_player_stats_player.head()



,season,player,position_x,age,Tm,games,GS,points,minutes_played,PER,...,DRB,rebounds,assists,steals,blocks,turnovers,PF,PTS.1,player_id,position_y
0,2017,Alex Abrines,SG,23,OKC,68,6,406,1055,10.1,...,68,86,40,37,8,33,114,406,0,SG
1,2017,Quincy Acy,PF,26,TOT,38,1,222,558,11.8,...,95,115,18,14,15,21,67,222,1,PF
2,2017,Quincy Acy,PF,26,DAL,6,0,13,48,-1.4,...,6,8,0,0,0,2,9,13,1,PF
3,2017,Quincy Acy,PF,26,BRK,32,1,209,510,13.1,...,89,107,18,14,15,19,58,209,1,PF
4,2017,Steven Adams,C,23,OKC,80,80,905,2389,16.5,...,333,615,86,88,78,146,195,905,2,C


In [10]:
nba_player_stats_team = pd.merge(nba_player_stats_player, pg_nba_teams, left_on='Tm', right_on='team_abbr')


nba_player_stats_team.head()

,season,player,position_x,age,Tm,games,GS,points,minutes_played,PER,...,PTS.1,player_id,position_y,team_abbr,team_city,team_name,team_id,nickname,state,year_founded
0,2017,Alex Abrines,SG,23,OKC,68,6,406,1055,10.1,...,406,0,SG,OKC,Oklahoma City,Oklahoma City Thunder,1610612760,Thunder,Oklahoma,1967
1,2017,Steven Adams,C,23,OKC,80,80,905,2389,16.5,...,905,2,C,OKC,Oklahoma City,Oklahoma City Thunder,1610612760,Thunder,Oklahoma,1967
2,2017,Semaj Christon,PG,24,OKC,64,1,183,973,5.7,...,183,80,PG,OKC,Oklahoma City,Oklahoma City Thunder,1610612760,Thunder,Oklahoma,1967
3,2017,Norris Cole,PG,28,OKC,13,0,43,125,5.4,...,43,83,PG,OKC,Oklahoma City,Oklahoma City Thunder,1610612760,Thunder,Oklahoma,1967
4,2017,Nick Collison,PF,36,OKC,20,0,33,128,12.8,...,33,85,PF,OKC,Oklahoma City,Oklahoma City Thunder,1610612760,Thunder,Oklahoma,1967


In [11]:
nba_player_stats_team.columns

Index(['season', 'player', 'position_x', 'age', 'Tm', 'games', 'GS', 'points',
       'minutes_played', 'PER', 'TS%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM',
       'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'eFG%', 'FT', 'FTA', 'free_throw_pc', 'ORB', 'DRB', 'rebounds',
       'assists', 'steals', 'blocks', 'turnovers', 'PF', 'PTS.1', 'player_id',
       'position_y', 'team_abbr', 'team_city', 'team_name', 'team_id',
       'nickname', 'state', 'year_founded'],
      dtype='object')

In [12]:
nba_player_stats=nba_player_stats_team[['player_id','team_id','season','age','games','points','minutes_played','free_throw_pc','rebounds','assists','steals','blocks','turnovers']]

nba_player_stats.head()


,player_id,team_id,season,age,games,points,minutes_played,free_throw_pc,rebounds,assists,steals,blocks,turnovers
0,0,1610612760,2017,23,68,406,1055,0.898,86,40,37,8,33
1,2,1610612760,2017,23,80,905,2389,0.611,615,86,88,78,146
2,80,1610612760,2017,24,64,183,973,0.548,88,130,28,6,43
3,83,1610612760,2017,28,13,43,125,0.800,11,14,8,0,7
4,85,1610612760,2017,36,20,33,128,0.625,30,12,2,2,4


In [13]:
con.execute("Truncate player_stats")

nba_player_stats.to_sql('player_stats', con, if_exists='append', index=False)

In [14]:
nba_teams2 = list(pg_nba_teams.team_id)

nba_teams2


[1610612737,
 1610612738,
 1610612739,
 1610612740,
 1610612741,
 1610612742,
 1610612743,
 1610612744,
 1610612745,
 1610612746,
 1610612747,
 1610612748,
 1610612749,
 1610612750,
 1610612751,
 1610612752,
 1610612753,
 1610612754,
 1610612755,
 1610612756,
 1610612757,
 1610612758,
 1610612759,
 1610612760,
 1610612761,
 1610612762,
 1610612763,
 1610612764,
 1610612765,
 1610612766]

In [15]:
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=nba_teams2)
games = pd.DataFrame(gamefinder.get_data_frames()[0])
games.head()
games.rename(columns={'GAME_ID':'game_id','GAME_DATE':'game_date','TEAM_ID':'team_id','MATCHUP':'matchup','WL':'result','FT_PCT':'free_throw_pc','REB':'rebounds','AST':'assists','STL':'steals','BLK':'blocks','TOV':'turnovers'}, inplace=True)


In [16]:

games_2017 = games[(games.SEASON_ID.str[-4:] == '2017') & (games.game_date < '2018-04-12') & (games.game_date > '2017-10-16')]

games_2017.head()




,SEASON_ID,team_id,TEAM_ABBREVIATION,TEAM_NAME,game_id,game_date,matchup,result,MIN,PTS,...,free_throw_pc,OREB,DREB,rebounds,assists,steals,blocks,turnovers,PF,PLUS_MINUS
102,22017,1610612737,ATL,Atlanta Hawks,0021701214,2018-04-10,ATL vs. PHI,L,240,113,...,0.692,8,31,39,27,10.0,3,10,24,-8.0
103,22017,1610612737,ATL,Atlanta Hawks,0021701202,2018-04-08,ATL @ BOS,W,239,112,...,0.786,5,32,37,23,7.0,4,13,19,6.0
104,22017,1610612737,ATL,Atlanta Hawks,0021701184,2018-04-06,ATL @ WAS,W,239,103,...,0.696,7,43,50,24,5.0,5,18,22,6.0
105,22017,1610612737,ATL,Atlanta Hawks,0021701172,2018-04-04,ATL vs. MIA,L,239,86,...,0.647,10,38,48,20,5.0,6,15,16,-29.0
106,22017,1610612737,ATL,Atlanta Hawks,0021701158,2018-04-03,ATL @ MIA,L,240,98,...,0.733,12,37,49,24,10.0,1,15,15,-3.0


In [17]:
games_2017_load = games_2017[['game_id','game_date','team_id','matchup','result','free_throw_pc','rebounds','assists','steals','blocks','turnovers']]

games_2017_load.head()

,game_id,game_date,team_id,matchup,result,free_throw_pc,rebounds,assists,steals,blocks,turnovers
102,0021701214,2018-04-10,1610612737,ATL vs. PHI,L,0.692,39,27,10.0,3,10
103,0021701202,2018-04-08,1610612737,ATL @ BOS,W,0.786,37,23,7.0,4,13
104,0021701184,2018-04-06,1610612737,ATL @ WAS,W,0.696,50,24,5.0,5,18
105,0021701172,2018-04-04,1610612737,ATL vs. MIA,L,0.647,48,20,5.0,6,15
106,0021701158,2018-04-03,1610612737,ATL @ MIA,L,0.733,49,24,10.0,1,15


In [18]:
con.execute("Truncate game_stats")

games_2017_load.to_sql('game_stats', con, if_exists='append', index=False)

In [19]:
con.close()